# Magic 5-gon Ring
Consider the following "magic" 3-gon ring, filled with the number $1$ to $6$, and each line adding to nine.</br>
!["not found"](./img/0068_1.png)
Working **clockwise**, and starting from the group of three with the numerically lowest external node ($4,3,2$ in this example), each solution can be described uniquely. For example, the above solution can be described by the set: $4,3,2;\, 6,2,1; \, 5,1,3$. </br>

It is possible to complete the ring with four different totals: $9,\,10,\,11,\text{ and } 12$. There are eight solutions in total. </br>
| **Total** | **Solution Set** |
|--- | ---|
| 9 | 4,2,3;  5,3,1;  6,1,2 |
| 9 | 4,3,2;  6,2,1;  5,1,3 |
|10 | 2,3,5;  4,5,1;  6,1,3 |
|10 | 2,5,3;  6,3,1;  4,1,5 |
|11 | 1,4,6;  3,6,2;  5,2,4 |
|11 | 1,6,4;  5,4,2;  3,2,6 |
|12 | 1,5,6;  2,6,4;  3,4,5 |
|12 | 1,6,5;  3,5,4;  2,4,6 |

By concatenating each group it is possible to form $9$-digit strings; the maximum string for a $3$-gon ring is $432621513$. </br>

Using the numbers $1$ to $10$, and depending on arrangements, it is possible to form $16$- and $17$-digit strings. What is the maximum $16$-digit for a "magic" $5$-gon ring?</br>
!["not found"](./img/0068_2.png)

In [1]:
import numpy as np

In [2]:
from itertools import combinations
from itertools import permutations

In [3]:
def all_elements_equal(arr):
    return np.all(arr == arr[0])

In [4]:
def cyclic_permutation(lst):
    N = len(lst)
    
    array = np.array(lst)
    
    a = np.argmin(array)
    
    result = []
    for i in range(N):
        result.append(lst[(a + i) % N])
    
    return result

In [5]:
def find_edge(digit_lst, sources):
    """
    Given external source, find corresponding edges
    
    Input : external sources (tuple)
    Output : [external source, (internal elements)]
    """
    result = []
    
    # External elements
    source_cases = list(permutations(sources))
    
    # Internal elements
    triangle = list(set(digit_lst) - set(sources))
    triangle_right = [(triangle[i], triangle[(i + 1) % 3]) for i in range(3)]
    triangle_reverse = [(triangle[i], triangle[(i + 2) % 3]) for i in range(3)]
    
    # Internal sum elements
    sum_right_lst = [sum(num) for num in triangle_right]
    sum_reverse_lst = [sum(num) for num in triangle_reverse]
    
    sum_lst = [sum_right_lst, sum_reverse_lst]
    
    # triangle right order
    for permutation in source_cases:
        edge = np.array(sum_right_lst) + np.array(permutation)
        if all_elements_equal(edge):
            edge_lst = []
            
            for i in range(len(permutation)):
                edge_lst.append([permutation[i], triangle_right[i]])
            
            result.append(edge_lst)
    
    # triangle reverse order
    for permutation in source_cases:
        edge = np.array(sum_reverse_lst) + np.array(permutation)
        if all_elements_equal(edge):
            edge_lst = []
            
            for i in range(len(permutation)):
                edge_lst.append([permutation[i], triangle_reverse[i]])
            
            result.append(edge_lst)
            
    return result

In [6]:
def three_gon(edges):
    """
    When lists are obtained from the find_edge, put them into the right order
    
    Input : edge obtained from find_edge()
    Output : edges in the right order
    """
    external_lst = []
    
    for edge in edges:
        external_lst.append(edge[0])
    
    right_order = cyclic_permutation(external_lst)
    
    sub_result = []
    
    for i in range(len(right_order)):
        target = right_order[i]
        
        for j in range(len(right_order)):
            if target == edges[j][0]:
                sub_result.append(edges[j])
                
    result = []
    
    for num_lst in sub_result:
        flattened = [item for sublist in num_lst for item in (sublist if isinstance(sublist, tuple) else [sublist])]
        for num in flattened:
            result.append(num)
    
    return result

In [7]:
# Solving 3-gon problem

digit_lst = [1, 2, 3, 4, 5, 6]

external_sources = list(combinations(digit_lst, 3))

result = []

for sources in external_sources:
    if find_edge(digit_lst, sources):
        for edge in find_edge(digit_lst, sources):
            result.append(three_gon(edge))
            
result

[[1, 5, 6, 2, 6, 4, 3, 4, 5],
 [1, 6, 5, 2, 4, 6, 3, 5, 4],
 [1, 4, 6, 3, 6, 2, 5, 2, 4],
 [1, 6, 4, 3, 2, 6, 5, 4, 2],
 [2, 3, 5, 4, 5, 1, 6, 1, 3],
 [2, 5, 3, 4, 1, 5, 6, 3, 1],
 [4, 2, 3, 5, 3, 1, 6, 1, 2],
 [4, 3, 2, 5, 1, 3, 6, 2, 1]]

# 5-gon ring

In [8]:
def separate_by_edge(internal):
    """
    Given internal elements in a 5-gon, pairing numbers.
    
    INPUT : internal number elements
    OUTPUT : pair list
    """
    l = len(internal)
    edges = []
    
    for i in range(l):
        edges.append([internal[i], internal[(i + 1) % l]])
        
    return edges

In [9]:
def judge_sum(external, internal_edges):
    """
    When we have external sources and internal edges obtained from separate_by_edge,
    by substituting these values, we obtain whether the sum of each edges are equal.
    
    INPUT : external sources, separate_by_edge(internal_sources)
    OUTPUT : boolean
    """
    
    l = len(external)
    sum_lst = []
    
    for i in range(l):
        sum_lst.append(external[i] + sum(internal_edges[i]))
    
    return all_elements_equal(sum_lst)

In [10]:
def permutation_except_cyclic(lst):
    perm_lst = list(permutations(lst))
    
    index = len(perm_lst) // len(lst)
    
    return perm_lst[:index]

In [11]:
def sum_edges(num_lst):
    l = len(num_lst)
    
    external = num_lst[: l // 2]
    internal = num_lst[l // 2 : l]
    
    if 10 in internal:
        return 0
    
    sum_lst = []
    
    for i in range(l // 2):
        sum_lst.append(external[i] + sum(separate_by_edge(internal)[i]))
        
    return sum_lst

In [12]:
from tqdm import tqdm
import time

In [13]:
def five_gon_edge(num_lst):
    l = len(num_lst)
    
    external = num_lst[: l//2]
    internal = num_lst[l//2 :]
    
    edge_pair = separate_by_edge(internal)
    
    edge_lst = []
    
    for i in range(l // 2):
        edge_lst.append([external[i], edge_pair[i]])
        
    return edge_lst

In [14]:
def edge_sum(edge):
    return edge[0] + sum(edge[1])

In [15]:
def edge_sums_equal(edge_lst):
    l = len(edge_lst)
    
    target_sum = edge_sum(edge_lst[0])
    
    for i in range(1, l):
        if not target_sum == edge_sum(edge_lst[i]):
            return False
    
    return True

In [44]:
def answer_to_string(answer):
    l = len(answer)
    external = []
    
    for edge in answer:
        external.append(edge[0])
    
    # find the minimum external
    k = external.index(min(external))
    
    ordered_answer = []
    
    for i in range(l):
        ordered_answer.append(answer[(i + k) % 5])
        
    result = ''
    
    for i in range(l):
        target = ordered_answer[i]
        
        result += str(target[0])
        result += str(target[1][0])
        result += str(target[1][1])
        
    return result

In [16]:
# Monte-Carlo Simulation

N = 10**(8)

digit_lst = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

answer_lst = []

start_time = time.time()

for i in tqdm(range(N)):
    np.random.shuffle(digit_lst)
    if 10 in digit_lst[5:]:
        continue
    else:
        edge_lst = five_gon_edge(digit_lst)
        if edge_sums_equal(edge_lst):
            answer_lst.append(edge_lst)
            
end_time = time.time()

execution_time = end_time - start_time
print(f"Execution time : {execution_time} seconds")

100%|████████████████████████████████████████████████| 100000000/100000000 [01:38<00:00, 1013338.23it/s]

Execution time : 98.70310401916504 seconds


In [17]:
answer_lst

[[[4, [9, 3]], [6, [3, 7]], [8, [7, 1]], [10, [1, 5]], [2, [5, 9]]],
 [[4, [9, 3]], [6, [3, 7]], [8, [7, 1]], [10, [1, 5]], [2, [5, 9]]],
 [[8, [2, 4]], [9, [4, 1]], [10, [1, 3]], [6, [3, 5]], [7, [5, 2]]],
 [[8, [2, 4]], [9, [4, 1]], [10, [1, 3]], [6, [3, 5]], [7, [5, 2]]],
 [[7, [5, 2]], [8, [2, 4]], [9, [4, 1]], [10, [1, 3]], [6, [3, 5]]],
 [[8, [4, 2]], [7, [2, 5]], [6, [5, 3]], [10, [3, 1]], [9, [1, 4]]],
 [[2, [9, 5]], [10, [5, 1]], [8, [1, 7]], [6, [7, 3]], [4, [3, 9]]],
 [[8, [7, 1]], [10, [1, 5]], [2, [5, 9]], [4, [9, 3]], [6, [3, 7]]],
 [[2, [5, 9]], [4, [9, 3]], [6, [3, 7]], [8, [7, 1]], [10, [1, 5]]],
 [[8, [4, 2]], [7, [2, 5]], [6, [5, 3]], [10, [3, 1]], [9, [1, 4]]],
 [[10, [3, 1]], [9, [1, 4]], [8, [4, 2]], [7, [2, 5]], [6, [5, 3]]],
 [[10, [5, 1]], [8, [1, 7]], [6, [7, 3]], [4, [3, 9]], [2, [9, 5]]],
 [[10, [1, 5]], [2, [5, 9]], [4, [9, 3]], [6, [3, 7]], [8, [7, 1]]],
 [[9, [1, 4]], [8, [4, 2]], [7, [2, 5]], [6, [5, 3]], [10, [3, 1]]],
 [[10, [3, 1]], [9, [1, 4]], [8, [

In [46]:
answer_string = []

for answer in answer_lst:
    target = answer_to_string(answer)
    
    if target in answer_string:
        continue
    else:
        answer_string.append(target)

In [47]:
answer_string

['2594936378711015',
 '6357528249411013',
 '6531031914842725',
 '2951051817673439']

In [48]:
max(answer_string)

'6531031914842725'